# Extract Chart Events from MIMIC III
Leveraging the variable list developed in "idClinicalVariables", I begin to extract the variables.

For each group of variables for which I am comparing similarity, I export a csv file. Each csv filename represents the category, subcategory, and datatype.  When generating the similarity matrices, I will read in each csv file, parse the type and generate 

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas_gbq
from google.oauth2 import service_account
import json
import pandas as pd
from functools import reduce
import re

In [8]:
# apply credentials
credentials = service_account.Credentials.from_service_account_file('../Patient-Similarity-credentials.json')
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "patient-similarity"

In [9]:
# load the identified clinical variables
with open('../data/clinical variables.txt', 'r') as f:
    clinical_variables = json.load(f)
    
clinical_variables['chart events']

{'physical assessment': ['Abdominal Assessment',
  'Skin Color',
  'Skin Condition',
  'Speech',
  'Gag Reflex',
  'Cough Reflex',
  'Oral Cavity',
  'Bowel Sounds',
  'Braden Moisture'],
 'activity': ['Activity',
  'Braden Mobility',
  'Braden Activity',
  'Activity Tolerance'],
 'pain': ['Pain Present',
  'Pain Location',
  'Pain Type',
  'Pain Level',
  'Pain Cause'],
 'diet': ['Braden Nutrition',
  'Diet Type',
  'Daily Weight',
  'Previous WeightF',
  'Admit Wt',
  'Previous Weight',
  'Appetite',
  'Weight Change',
  'Special diet'],
 'demographics': ['Marital Status', 'Family Communication', 'Gender', 'Race'],
 'heart': ['Heart Rate', 'Heart Rhythm'],
 'lung': ['Respiratory Rate',
  'RUL Lung Sounds',
  'LLL Lung Sounds',
  'RLL Lung Sounds',
  'LUL Lung Sounds',
  'Respiratory Pattern',
  'Respiratory Effort'],
 'medical history': ['Past medical history',
  'CV - past medical history',
  'Mental status',
  'Recreational drug use']}

In [11]:
# import our variable mapping
from variableMaps.chartEventsMap import *
chartMap

{'Occ. Moist': 'Occasionally Moist',
 'Consist. Moist': 'Consistently Moist',
 'Intubated/trach': 'Intubated/trached',
 'Walks Occasional': 'Walks Occasionally',
 'Comp. Immobile': 'Completely Immobile',
 'Cough/DeepBreath': 'Cough/Deep Breath',
 'Prob. Inadequate': 'Probably Inadequate',
 'Clear liquid': 'Clear Liquid',
 'House - Regular': 'House',
 'NAS/Low Cholest': 'NAS/Low Cholesterol',
 'Sl. Limited': 'Slight Limitations',
 'Fam Talked to RN': 'Family Talked to RN',
 'Fam Talked to MD': 'Family Talked to MD',
 'Family Conferenc': 'Family Conference',
 'SocServ Involved': 'Social Service Involved',
 'S': 'Single',
 'M': 'Married',
 'W': 'Widowed',
 'Partner': 'Married',
 'D': 'Divorced',
 's': 'Single',
 'White - Other European': 'White',
 'White - Russian': 'White',
 'White - Brazilian': 'White',
 'Portuguese': 'White',
 'American Indian / Alaska Native - Federally Recognized Tribe': 'American Indian / Alaska Native',
 'Hispanic / Latino - Salvadoran': 'Hispanic or Latino',
 'His

In [12]:
# since I have to fix some of the data issues in python, I need to get frequencies in python
def extractFrequencies(df):
    # first group the newly cleaned data
    df = df.groupby(['subject_id','label', 'value']).sum().reset_index()
    
    # now compute frequencies
    total_counts = df.groupby(['subject_id','label']).sum().reset_index()
    total_counts.columns = ['subject_id', 'label', 'total_counts']
    df = pd.merge(df, total_counts, on = ['subject_id', 'label'], how = 'left')
    df['frequency'] = df.num_occurences/df.total_counts
    return df

## Cross Sectional Data
Extract data for each group: Physical assessment, activity, pain, diet, demographics, heart, lung, and patient medical history.

we use multiple correspondence analysis to convert categorical to numeric  

### Physical Assessment

In [4]:
phys_assess = clinical_variables['chart events']['physical assessment']
phys_assess

['Abdominal Assessment',
 'Skin Color',
 'Skin Condition',
 'Speech',
 'Gag Reflex',
 'Cough Reflex',
 'Oral Cavity',
 'Bowel Sounds',
 'Braden Moisture']

In [20]:
# Get the frequency of each label and value for each patient
q = """SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Abdominal Assessment','Skin Color', 'Skin Condition', 'Speech',
 'Gag Reflex', 'Cough Reflex', 'Oral Cavity', 'Bowel Sounds', 'Braden Moisture')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences
"""

phys_assess = pandas_gbq.read_gbq(q)
phys_assess.head()

,subject_id,label,value,num_occurences
0,4,Abdominal Assessment,Soft,5
1,4,Bowel Sounds,Present,5
2,4,Braden Moisture,Occ. Moist,2
3,4,Braden Moisture,Rarely Moist,4
4,4,Oral Cavity,Teeth/Tissue WNL,9


In [25]:
# clean our data and extract frequencies
phys_assess = phys_assess.replace({"value":chartMap})
phys_assess = extractFrequencies(phys_assess)
phys_assess.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Abdominal Assessment,Soft,5,5,1.000000
1,4,Bowel Sounds,Present,5,5,1.000000
2,4,Braden Moisture,Occasionally Moist,2,6,0.333333
3,4,Braden Moisture,Rarely Moist,4,6,0.666667
4,4,Oral Cavity,Teeth/Tissue WNL,9,9,1.000000


In [26]:
phys_assess.to_csv("../data/patientData/chart events_categorical_physical assessment.csv", index = False)

## Activity

In [13]:
activity = clinical_variables['chart events']['activity']
activity

['Activity', 'Braden Mobility', 'Braden Activity', 'Activity Tolerance']

In [14]:
q = """
SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Activity', 'Braden Mobility', 'Braden Activity', 'Activity Tolerance')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences"""
activity = pandas_gbq.read_gbq(q)
activity.head()

,subject_id,label,value,num_occurences
0,4,Activity,Ambulate,1
1,4,Activity,Bedrest,5
2,4,Activity,Commode,4
3,4,Activity Tolerance,Good,3
4,4,Activity Tolerance,Tolerated Well,7


In [15]:
activity = activity.replace({"value":chartMap})
activity = extractFrequencies(activity)
activity.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Activity,Ambulate,1,10,0.1
1,4,Activity,Bedrest,5,10,0.5
2,4,Activity,Commode,4,10,0.4
3,4,Activity Tolerance,Good,3,10,0.3
4,4,Activity Tolerance,Tolerated Well,7,10,0.7


In [16]:
activity.to_csv("../data/patientData/chart events_categorical_activity.csv", index = False)

## Pain

In [18]:
pain = clinical_variables['chart events']['pain']
pain

['Pain Present', 'Pain Location', 'Pain Type', 'Pain Level', 'Pain Cause']

In [19]:
# we are going to pain a little differently
# cross-sectional of pain descriptions (location, cause, present, and type)
q = """SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Pain Present', 'Pain Location', 'Pain Type', 'Pain Cause')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences"""
pain = pandas_gbq.read_gbq(q)
pain.head()

,subject_id,label,value,num_occurences
0,4,Pain Location,Not Indicated,8
1,4,Pain Present,No,11
2,52,Pain Cause,At Rest,2
3,52,Pain Location,Abdominal,1
4,52,Pain Location,Headache,5


In [20]:
pain = pain.replace({"value":chartMap})
pain = extractFrequencies(pain)
pain.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Pain Location,Not Indicated,8,8,1.000000
1,4,Pain Present,No,11,11,1.000000
2,52,Pain Cause,At Rest,2,2,1.000000
3,52,Pain Location,Abdominal,1,9,0.111111
4,52,Pain Location,Headache,5,9,0.555556


In [19]:
pain.to_csv("../data/patientData/chart events_categorical_pain.csv", index = False)

In [34]:
# now we separate pain level.  we make the columns the location of the pain, and we average out the pain for that location
q = """SELECT A.subject_id, A.CHARTTIME as date, 
max(case when B.LABEL = "Pain Location" then A.VALUE else null end) as location,
max(case when B.LABEL = "Pain Level" then A.VALUE else null end) as level
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Pain Location', 'Pain Level')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%" and lower(VALUE) != "unable to score"
group by subject_id, charttime
having location is not null and level is not null
order by subject_id, A.CHARTTIME """
pain_level = pandas_gbq.read_gbq(q)
pain_level.head()

,subject_id,date,location,level
0,52,2191-01-10 04:00:00,Headache,10-Worst
1,52,2191-01-10 08:00:00,Headache,10-Worst
2,52,2191-01-10 16:00:00,Headache,5-Moderate
3,52,2191-01-10 20:00:00,Abdominal,2-Mild
4,117,2133-11-18 23:00:00,Back,6-Mod to Severe


In [35]:
# convert to ordinal
pain_level = pain_level.replace({"level":painLevelMap})
pain_level.head()

,subject_id,date,location,level
0,52,2191-01-10 04:00:00,Headache,10
1,52,2191-01-10 08:00:00,Headache,10
2,52,2191-01-10 16:00:00,Headache,5
3,52,2191-01-10 20:00:00,Abdominal,2
4,117,2133-11-18 23:00:00,Back,6


In [36]:
# now we take the average by location and transpose
pain_level = pain_level.groupby(['subject_id', 'location'], as_index=False).mean()\
                    .pivot(index='subject_id', columns='location', values='level')
del pain_level.columns.name
pain_level = pain_level.fillna(0)
pain_level = pain_level.reset_index()
pain_level.head()

,subject_id,Abdominal,Back,Both Legs,Chest Pain,Generalized,Headache,Incisional,Jaw,Left Arm,...,Right Elbow,Right Flank,Right Foot,Right Hip,Right Leg,Right Lower Quad,Right Lower Quadrant,Right Shoulder,Right Upper Quad,Right Upper Quadrant
0,52,2.000000,0.0,0.0,0.0,0.0,8.333333,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,117,0.000000,5.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,143,7.307692,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,172,0.000000,6.5,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,188,5.750000,6.0,0.0,0.0,0.0,0.000000,6.111111,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,6.0,0.0


In [37]:
pain_level.to_csv("../data/patientData/chart events_numeric_pain level.csv", index = False)

## Diet and Nutrition

In [32]:
diet = clinical_variables['chart events']['diet']
diet

['Braden Nutrition',
 'Diet Type',
 'Daily Weight',
 'Previous WeightF',
 'Admit Wt',
 'Previous Weight',
 'Appetite',
 'Weight Change',
 'Special diet']

In [33]:
# categorical and numeric.  for numeric just doing max weight, min weight, and change
q = """
SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Braden Nutrition','Diet Type', 'Appetite','Special diet')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences"""
diet = pandas_gbq.read_gbq(q)
diet.head()

,subject_id,label,value,num_occurences
0,4,Braden Nutrition,Excellent,3
1,4,Braden Nutrition,Prob. Inadequate,3
2,4,Diet Type,Diabetic,8
3,52,Braden Nutrition,Prob. Inadequate,7
4,52,Braden Nutrition,Very Poor,1


In [35]:
diet = diet.replace({"value":chartMap})
diet = extractFrequencies(diet)
diet.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Braden Nutrition,Excellent,3,6,0.500
1,4,Braden Nutrition,Probably Inadequate,3,6,0.500
2,4,Diet Type,Diabetic,8,8,1.000
3,52,Braden Nutrition,Probably Inadequate,7,8,0.875
4,52,Braden Nutrition,Very Poor,1,8,0.125


In [36]:
diet.to_csv("../data/patientData/chart events_categorical_diet.csv", index = False)

In [37]:
# now for the numeric
q = """SELECT A.subject_id, max(A.ValueNum) as max_wgt, min(A.ValueNum) as min_wgt, 
  max(A.ValueNum) - min(A.ValueNum) as wgt_change, 
  (max(A.ValueNum) - min(A.ValueNum))/max(A.ValueNum) as loss_perc
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Daily Weight','Previous WeightF','Admit Wt','Previous Weight')
and A.HADM_ID is not null and A.VALUE is not null
and A.ValueNum > 0
group by subject_id
order by subject_id """
weight = pandas_gbq.read_gbq(q)
weight.head()

,subject_id,max_wgt,min_wgt,wgt_change,loss_perc
0,4,53.599998,53.599998,0.000000,0.00000
1,52,81.300003,81.300003,0.000000,0.00000
2,78,72.400002,72.400002,0.000000,0.00000
3,117,108.099998,79.500000,28.599998,0.26457
4,143,130.500000,113.000000,17.500000,0.13410


In [38]:
weight.to_csv("../data/patientData/chart events_numeric_weight.csv", index = False)

## Demographics

In [8]:
demographics = clinical_variables['chart events']['demographics']
demographics

['Marital Status', 'Family Communication', 'Gender', 'Race']

In [9]:
# I need to merge categorical data from chart events and patients
# I then treat age as a numeric variable and it is excluded from the MCA
q = """SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Marital Status', 'Family Communication', 'Race')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences"""
demographics = pandas_gbq.read_gbq(q)
demographics.head()

,subject_id,label,value,num_occurences
0,4,Family Communication,Family Called,1
1,4,Family Communication,Family Visited,2
2,4,Marital Status,S,1
3,52,Family Communication,Fam Talked to MD,1
4,52,Family Communication,Family Visited,4


In [10]:
demographics = demographics.replace({"value":chartMap})
demographics = extractFrequencies(demographics)
demographics.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Family Communication,Family Called,1,3,0.333333
1,4,Family Communication,Family Visited,2,3,0.666667
2,4,Marital Status,Single,1,1,1.000000
3,52,Family Communication,Family Talked to MD,1,5,0.200000
4,52,Family Communication,Family Visited,4,5,0.800000


In [11]:
# Now we extract age and gender from the patient table 
q = """
select A.subject_id, A.gender, date_diff(Date(B.last_date), Date(A.dob), YEAR) as age, 
from `patient-similarity.mimic.patients` as A
left join (
  SELECT subject_id, max(dischtime) as last_date FROM `patient-similarity.mimic.admissions` 
  group by subject_id
) as B
using(subject_id)
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
order by subject_id"""
age_gender = pandas_gbq.read_gbq(q)
age_gender.head()

,subject_id,gender,age
0,4,F,48
1,52,M,39
2,78,M,49
3,117,F,50
4,140,M,53


In [12]:
# format gender to match categorical data
gender = age_gender.melt(id_vars = "subject_id")
gender.columns = ["subject_id", "label", "value"]
gender = gender[gender['label']== "gender"]
gender['num_occurences'] = 1
gender['total_counts'] = 1
gender['frequency'] = 1
gender.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,gender,F,1,1,1
1,52,gender,M,1,1,1
2,78,gender,M,1,1,1
3,117,gender,F,1,1,1
4,140,gender,M,1,1,1


In [13]:
demographics = pd.concat([demographics, gender], axis = 0, sort = False).sort_values("subject_id")
demographics.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Family Communication,Family Called,1,3,0.333333
2,4,Marital Status,Single,1,1,1.000000
0,4,gender,F,1,1,1.000000
1,4,Family Communication,Family Visited,2,3,0.666667
3,52,Family Communication,Family Talked to MD,1,5,0.200000


In [14]:
demographics.to_csv("../data/patientData/chart events_categorical_demographics.csv", index = False)

In [15]:
# export age as numeric
age = age_gender.drop(columns = "gender")
age.head()

,subject_id,age
0,4,48
1,52,39
2,78,49
3,117,50
4,140,53


In [17]:
age.to_csv("../data/patientData/chart events_numeric_demographics.csv", index = False)

## Heart and Lung
Combining heart and lung measurements and following the same format as diet. Numeric columns are separated and will be added back later 

In [65]:
heart = clinical_variables['chart events']['heart']
lung = clinical_variables['chart events']['lung']
heart_lung = heart + lung
heart_lung

['Heart Rate',
 'Heart Rhythm',
 'Respiratory Rate',
 'RUL Lung Sounds',
 'LLL Lung Sounds',
 'RLL Lung Sounds',
 'LUL Lung Sounds',
 'Respiratory Pattern',
 'Respiratory Effort']

In [93]:
# first we extract the categorical columns#
q = """SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Heart Rhythm', 'RUL Lung Sounds', 'LLL Lung Sounds', 'RLL Lung Sounds',
 'LUL Lung Sounds', 'Respiratory Pattern', 'Respiratory Effort')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences"""
heart_lung = pandas_gbq.read_gbq(q)
heart_lung.head()

,subject_id,label,value,num_occurences
0,4,Heart Rhythm,Normal Sinus,26
1,4,Heart Rhythm,Sinus Tachy,4
2,4,LLL Lung Sounds,Clear,1
3,4,LLL Lung Sounds,Crackles,4
4,4,LLL Lung Sounds,Diminished,4


In [94]:
heart_lung = heart_lung.replace({"value":chartMap})
heart_lung = extractFrequencies(heart_lung)
heart_lung.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Heart Rhythm,Normal Sinus,26,30,0.866667
1,4,Heart Rhythm,ST (Sinus Tachycardia),4,30,0.133333
2,4,LLL Lung Sounds,Clear,1,9,0.111111
3,4,LLL Lung Sounds,Crackles,4,9,0.444444
4,4,LLL Lung Sounds,Diminished,4,9,0.444444


In [97]:
heart_lung.to_csv("../data/patientData/chart events_categorical_heart lung.csv", index = False)

In [99]:
# Now for numeric - min, max, difference, average for heart rate and respiratory rate
q = """SELECT A.subject_id, 
max(case when B.LABEL = "Heart Rate" then A.ValueNum else null end) as max_heart_rate, 
min(case when B.LABEL = "Heart Rate" then A.ValueNum else null end) as min_heart_rate, 
avg(case when B.LABEL = "Heart Rate" then A.ValueNum else null end) as avg_heart_rate, 

max(case when B.LABEL = 'Respiratory Rate' then A.ValueNum else null end) as max_resp_rate, 
min(case when B.LABEL = 'Respiratory Rate' then A.ValueNum else null end) as min_resp_rate, 
avg(case when B.LABEL = 'Respiratory Rate' then A.ValueNum else null end) as avg_resp_rate, 


FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Heart Rate','Respiratory Rate')
and A.HADM_ID is not null and A.VALUE is not null
and A.ValueNum > 0
group by subject_id
order by subject_id """
heart_lung_rate = pandas_gbq.read_gbq(q)
heart_lung_rate.head()

,subject_id,max_heart_rate,min_heart_rate,avg_heart_rate,max_resp_rate,min_resp_rate,avg_resp_rate
0,4,111.0,74.0,90.354839,32.0,18.0,25.333333
1,52,106.0,85.0,94.240000,27.0,11.0,17.607843
2,78,86.0,56.0,64.681818,24.0,11.0,17.000000
3,117,219.0,56.0,83.629191,34.0,9.0,21.513889
4,140,117.0,65.0,84.533333,28.0,8.0,18.911111


In [100]:
heart_lung_rate.to_csv("../data/patientData/chart events_numeric_heart lung rate.csv", index = False)

## Medical History

In [7]:
med_history = clinical_variables['chart events']['medical history']
med_history

['Past medical history',
 'CV - past medical history',
 'Mental status',
 'Recreational drug use']

In [8]:
# I need to convert the values of the medical history values to be the label and then the value can take on 0 or 1
# I just do it with two queries, and concatenate.  
# I then need to fillna with 0 for indicator matrix on this one
q = """
SELECT distinct A.subject_id, B.label, A.value, count(A.Value) as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Mental status', 'Recreational drug use')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, LABEL, VALUE
order by Subject_ID, LABEL, VALUE, num_occurences"""
mental_drug = pandas_gbq.read_gbq(q)
mental_drug.head()

,subject_id,label,value,num_occurences
0,188,Mental status,Forgets limitations,45
1,188,Mental status,Oriented to own ability,3
2,188,Recreational drug use,0,11
3,236,Mental status,Oriented to own ability,8
4,236,Recreational drug use,0,1


In [15]:
mental_drug = extractFrequencies(mental_drug)
mental_drug.head()

,subject_id,label,value,num_occurences,total_counts,frequency
0,188,Mental status,Forgets limitations,45,48,0.9375
1,188,Mental status,Oriented to own ability,3,48,0.0625
2,188,Recreational drug use,0,11,11,1.0000
3,236,Mental status,Oriented to own ability,8,8,1.0000
4,236,Recreational drug use,0,1,1,1.0000


In [16]:
mental_drug.to_csv("../data/patientData/chart events_categorical_mental drug.csv", index = False)

In [29]:
q = """
SELECT distinct A.subject_id, A.value as label, 1 as values, 1 as num_occurences
FROM `patient-similarity.mimic.chartevents` as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and LABEL in ('Past medical history', 'CV - past medical history')
and A.HADM_ID is not null and A.VALUE is not null
and lower(VALUE) not like "%other%"
group by Subject_ID, VALUE
order by Subject_ID, VALUE, num_occurences"""
med_history = pandas_gbq.read_gbq(q)
med_history.columns = ['subject_id', 'label', 'value', 'num_occurences']
med_history.head()

,subject_id,label,value,num_occurences
0,188,Diabetes - Insulin,1,1
1,188,Diabetes - Oral Agent,1,1
2,188,GI Bleed,1,1
3,188,HEMO or PD,1,1
4,188,Hepatitis,1,1


In [30]:
med_history = pd.pivot(med_history, index = "subject_id", columns = "label", values = "value")
med_history = med_history.fillna(0)
med_history.columns = [re.sub("\\s+", "_", c).lower() for c in med_history.columns]
del med_history.columns.name
med_history = med_history.reset_index()
med_history.head()

,subject_id,anemia,angina,arrhythmias,asthma,cad,chf,copd,cva,diabetes_-_insulin,...,hepatitis,hypertension,liver_failure,mi,pvd,pacemaker,pancreatitis,renal_failure,seizures,smoker
0,188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,357,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,422,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,689,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [31]:
med_history.to_csv("../data/patientData/chart events_numeric_medical history.csv", index = False)